In [248]:
import xml.etree.ElementTree as ET
import pandas as pd

# xml파일 파싱해서 contestuid를 key로 하고 문자열을 value로 하는 사전을 리턴
# contestuid가 중복이 있을 경우 그에 따른 메세지 도출

def parse_xml_to_dict(script_dir):
    result_dict = {}
    tree = ET.parse(script_dir)
    root = tree.getroot()
    
    # root = ET.fromstring(xml_content)
    for content_elem in root.findall('content'):
        contentuid = content_elem.get('contentuid')
        content_text = content_elem.text
        if result_dict.get(contentuid) == None:
            result_dict[contentuid] = content_text
        else:
            print("Duplicate Key!!", contentuid, content_text)
        
    return result_dict

def detect_language(text):
    total_chars = len(text)

    korean_count = sum(1 for char in text if 0xAC00 <= ord(char) <= 0xD7A3)

    return korean_count / total_chars #>= 0.5

userTranslate_path_0815 = "../train_data/BG3(raw)/ht_ko_en.xml"
userTranslate_path_0819 = "../train_data/BG3(raw)/0819.xml"
userTranslate_path_0820 = "../train_data/BG3(raw)/0820.xml"
english_path = "../train_data/BG3(raw)/en.xml"

userTranslate_dict_0815 = parse_xml_to_dict(userTranslate_path_0815)
userTranslate_dict_0819 = parse_xml_to_dict(userTranslate_path_0819)
english_dict = parse_xml_to_dict(english_path)

# english_corp = []
# korean_corp = []
# target_key = []
# korean_key = []

# for key in english_dict:
#     if detect_language(userTranslate_dict[key]) >= 0.4:
#         korean_corp.append(userTranslate_dict[key])
#         english_corp.append(english_dict[key])
#         korean_key.append(key)
#     else:
#         target_key.append(key)

# print(len(korean_corp))
# print(len(english_corp))
# print(len(english_dict))
# print(len(userTranslate_dict))

test_key = []
target_dict = {}

for key in english_dict:
    if detect_language(userTranslate_dict_0815[key]) <= 0.05:
        if detect_language(userTranslate_dict_0819[key]) >= 0.3:
            test_key.append(key)

for key in english_dict:
    if not detect_language(userTranslate_dict_0819[key]) >= 0.3:
        if target_dict.get(key) == None:
            target_dict[key] = english_dict[key]
        else:
            print("Duplicate Key!!", key)


print(len(target_dict))
print(len(test_key))

163512
5957


In [2]:
print(len(target_dict))
target_dict_reverse = dict(map(reversed, target_dict.items()))

163512


In [3]:
target_corp = []
# for key in target_dict:
#     if not len(target_dict[key]) <=2:
#         target_corp.append(target_dict[key])
# print(len(target_corp))

for key in target_dict:
    target_corp.append(target_dict[key])
print(len(target_corp))

163512


In [4]:
for i in target_corp:
    if len(i) <= 2:
        print(i)

6
J
I
0
|
AI
F8
, 
N
F
SP
HA
|
UI
No
.
1
,
No
S
!
2
5
F4
Us
3
Me
F3
HP
F1
K
R
C
U
On
E
T
9
O
x1
|
Y
F7
W
=
-
|
: 
.
Q
B
No
7
AC
E
8
Z
D
A
4
/
H
No
G
F6
X
|
No
F2
M
P
AP
F5
F9
L
V


In [5]:
target_dict_reverse[target_corp[23456]]

'h25a3a48dg1f09g48dbgbdf6g5077a72613c3'

In [6]:
from vllm import LLM, SamplingParams

sampling_params = SamplingParams(
    temperature=0.001,
    max_tokens=2048,
    top_p=0.95,
    stop="</끝>"
    )

llm = LLM(model="aeolian83/Gugugo_for_DnD_v0.8", 
gpu_memory_utilization=0.9,
dtype='bfloat16')

WARNING 08-21 17:59:55 config.py:284] Casting torch.float16 to torch.bfloat16.
INFO 08-21 17:59:55 llm_engine.py:70] Initializing an LLM engine with config: model='aeolian83/Gugugo_for_DnD_v0.8', tokenizer='aeolian83/Gugugo_for_DnD_v0.8', tokenizer_mode=auto, trust_remote_code=False, dtype=torch.bfloat16, use_dummy_weights=False, download_dir=None, use_np_weights=False, tensor_parallel_size=1, seed=0)
INFO 08-21 18:00:08 llm_engine.py:196] # GPU blocks: 6242, # CPU blocks: 1365


In [7]:
prompts = []
for i in target_corp:
    prompts.append(f"### 영어: {i}</끝>\n### 한국어:")

print(len(prompts))

163512


In [8]:
print(prompts[13891])
print(prompts[13892])
print(prompts[13896])
print(prompts[13908])

### 영어: On a hit, inflict [1] turn of <LSTag Type="Status" Tooltip="MAG_RADIANT_RADIATING_ORB">Radiating Orb</LSTag> upon the target.</끝>
### 한국어:
### 영어: Basket is gonna MURDERISE you.</끝>
### 한국어:
### 영어: When I conquer the world over, perhaps I shall spare you.</끝>
### 한국어:
### 영어: Energy Source</끝>
### 한국어:


In [9]:
outputs = llm.generate(prompts, sampling_params)

Processed prompts:  71%|███████▏  | 116739/163512 [26:15<12:38, 61.63it/s] 

WARNING 08-21 18:26:38 scheduler.py:130] Input prompt (9603 tokens) is too long and exceeds limit of 2048


Processed prompts: 100%|██████████| 163512/163512 [38:07<00:00, 71.48it/s] 


In [10]:
prompts[:4]

["### 영어: Sorry, darling, I haven't got time for underlings.</끝>\n### 한국어:",
 '### 영어: <LSTag Tooltip="MovementSpeed">Movement Speed</LSTag>: [1]/[2]</끝>\n### 한국어:',
 "### 영어: If that's her 'formula' I can smell, it's even fouler than her blood. Gods below...</끝>\n### 한국어:",
 "### 영어: *Thus you shall spend the rest of your days rutting, the lash of Father's disappointment the only thought in your rat's mind.*</끝>\n### 한국어:"]

In [11]:
print(outputs[0].outputs[0].text[1:])
print(outputs[1].outputs[0].text[1:])
print(outputs[2].outputs[0].text[1:])
print(outputs[3].outputs[0].text[1:])

미안하구나, 자기야, 난 부하들을 기다릴 시간이 없어.
<LSTag Tooltip="MovementSpeed">이동 속도</LSTag>: [1]/[2]
그게 그녀의 '처방'이라면, 냄새가 나는군, 그녀의 피보다 더 더러운 냄새가 나는군. 신들은 아래에...
*그래서 당신은 하루 종일 쥐를 잡는 일을 하게 됩니다. 당신의 쥐의 마음 속에 있는 아버지의 실망스러운 손길에는 오직 한 가지 생각밖에 없습니다.*


In [12]:
result_dict = {}

for i, key in enumerate(target_dict):
    if result_dict.get(key) == None:
        result_dict[key] = outputs[i].outputs[0].text[1:]
    else:
        print("error")

In [13]:
result_dict

{'h000006d4gcefbg4092gbb39gfeb27a3bb0a7': '미안하구나, 자기야, 난 부하들을 기다릴 시간이 없어.',
 'h00010f77gdae2g459eg9e30g36892aea6b2d': '<LSTag Tooltip="MovementSpeed">이동 속도</LSTag>: [1]/[2]',
 'h00011ac6ge2f4g4104gbc6fg246521189b73': "그게 그녀의 '처방'이라면, 냄새가 나는군, 그녀의 피보다 더 더러운 냄새가 나는군. 신들은 아래에...",
 'h0001dc8ag4fd9g4d8ag8775ga6313d6a7ff4': '*그래서 당신은 하루 종일 쥐를 잡는 일을 하게 됩니다. 당신의 쥐의 마음 속에 있는 아버지의 실망스러운 손길에는 오직 한 가지 생각밖에 없습니다.*',
 'h0001eb83ge642g48eegaceeg497c92c8b8d3': '울마',
 'h00022debgbe37g460cg83a5g28893d382c44': '처음으로 그녀가 처음으로 말을 걸었을 때부터 먹기 시작했을 때부터 먹기 시작했어. 내 이름은 한 마디도 못 들었어.',
 'h0002ca0fg57a0g4d61g8475gf10d818fe137': '포자 확산',
 'h000347eag76d8g47b6g9fe3g4f2162783f9d': '그 일은 성공했어.',
 'h00039766g9af0g496eg8024g3b3da11cdde5': '*첫 번째 지옥의 불이 당신을 향해 맹렬히 타오르고 있습니다.*',
 'h0003edd4g9ae0g4246gbf0dg0c404ffc6585': '*당신은 술을 마시고 또 마시고 또 마시고. 그 순간, 울음조차 나오지 않습니다.*',
 'h0003efc9ga242g4635ga4f7g4e5dab77103b': '그래. 우리 엉덩이는.',
 'h0003f45dg0cd8g4cccg966fg766f3949baf8': '<LSTag Tooltip="CriticalHit">공격 적중</LSTag>을 성공한 후, 추가

In [14]:
import xml.etree.ElementTree as ET
from tqdm import tqdm

def modify_content_by_uid(xml_file, contentuid, new_content):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for content_elem in root.iter('content'):
        if content_elem.get('contentuid') == contentuid:
            content_elem.text = new_content
            break

    tree.write(xml_file)

In [15]:
# h5684b0e3g5adbg4eaag97fcg010b4dc21f36

result_dict['h1f4a0dd2g72d8g4ce0g9d04ge965d2e536ce'] = english_dict['h1f4a0dd2g72d8g4ce0g9d04ge965d2e536ce']
result_dict['h5684b0e3g5adbg4eaag97fcg010b4dc21f36'] = english_dict['h5684b0e3g5adbg4eaag97fcg010b4dc21f36']
print(result_dict['h1f4a0dd2g72d8g4ce0g9d04ge965d2e536ce'])
print(result_dict['h5684b0e3g5adbg4eaag97fcg010b4dc21f36'])

BLOOD AND BODILY FLUID LOG<br><br>Addendum the First: It is no longer necessary to log names. Race or creature type is sufficient. - MT<br><br>Adendum the Second: You will use either the term 'donor' or 'vessel' when describing a source of extraction. 'Victim' is an ugly and inaccurate word.<br><br>Name: [REDACTED]<br>Race: Half-Elf<br>Occupation: Farmer<br>Cause of Death: Extremity amputation<br>Extraction Fluid: Blood<br>Notes: Cadaver dumped in canal<br><br>Name: [REDACTED]<br>Race: Human<br>Occupation: Unknown<br>Cause of Death: Live extraction<br>Extraction Fluid: Blood, lymph, spinal fluid<br>Notes: Refused toll payment. Cadaver cremated after extraction.<br><br>Name: UNKNOWN<br>Race: Ogre<br>Occupation:<br>Cause of Death: Fatal wounds to skull and torso<br>Extraction fluid: Blood<br>Notes: Corpse confiscated by Balthazar<br><br>[The list continues for many pages.]
[Scrawled records of guests cover pages stained with ink, grease, and gods-know what else.] <br><br>Name: Bernyr<br>

In [249]:
userTranslate_dict_0820 = parse_xml_to_dict(userTranslate_path_0820)

In [263]:
xml_file_path = "../train_data/BG3(raw)/result_0820.xml"

tree = ET.parse(xml_file_path)
root = tree.getroot()

for content_elem in root.iter('content'):
    id = content_elem.get('contentuid')
    text = content_elem.text
    if detect_language(text) >= 0.25:
        continue
    if result_dict.get(id) == None:
        continue
    content_elem.text = result_dict[id]

tree.write(xml_file_path, encoding="utf-8")
print("Done!!!")

Done!!!


In [17]:
# import pickle

# user = {'name':'Andrew K. Johnson', 'score': 199, 'location':[38.189323, 127.3495672]}

# # save data
# with open('user.pickle','wb') as fw:
#     pickle.dump(user, fw)

# # load data
# with open('user.pickle', 'rb') as fr:
#     user_loaded = pickle.load(fr)

# # show data
# print(user_loaded)

In [43]:
# import pickle

# save data
# with open('../test_data/result_dict.pickle','wb') as fw:
#     pickle.dump(result_dict, fw)

# with open('../test_data/result_dict.pickle', 'rb') as fr:
#     result_dict = pickle.load(fr)


In [261]:
print(result_dict['h973dd6eag77d1g41ddg9a3fg45b16106c9af'])
print(english_dict['h973dd6eag77d1g41ddg9a3fg45b16106c9af'])

The Aspect of the Crocodile increases your agility in the water.<br><br>Your <LSTag Tooltip="MovementSpeed">movement speed</LSTag> increases by [1] while standing in water-based surfaces.<br><br>On slippery surfaces, you also have <LSTag Tooltip="Advantage">Advantage</LSTag> on <LSTag Tooltip="SavingThrow">Saving Throws</LSTag> against being knocked <LSTag Type="Status" Tooltip="PRONE">Prone</LSTag>.
The Aspect of the Crocodile increases your agility in the water.<br><br>Your <LSTag Tooltip="MovementSpeed">movement speed</LSTag> increases by [1] while standing in water-based surfaces.<br><br>On slippery surfaces, you also have <LSTag Tooltip="Advantage">Advantage</LSTag> on <LSTag Tooltip="SavingThrow">Saving Throws</LSTag> against being knocked <LSTag Type="Status" Tooltip="PRONE">Prone</LSTag>.


In [260]:
result_dict['h973dd6eag77d1g41ddg9a3fg45b16106c9af'] = english_dict['h973dd6eag77d1g41ddg9a3fg45b16106c9af']

In [262]:
# with open('../test_data/result_dict_clean.pickle','wb') as fw:
#     pickle.dump(result_dict, fw)

In [19]:
# userTranslate_dict_0815
# userTranslate_dict_0819
# english_dict
# test_key

# test_dict = {}
# for key in test_key:
#     test_dict[key] = english_dict[key]

In [20]:
# test_corp = []
# for key in test_dict:
#     test_corp.append(test_dict[key])

# prompts = []
# for i in test_corp:
#     prompts.append(f"### 영어: {i}</끝>\n### 한국어:")

# print(len(prompts))

In [21]:
# outputs = llm.generate(prompts, sampling_params)

In [22]:
# result_dict = {}

# for i, key in enumerate(test_dict):
#     if result_dict.get(key) == None:
#         result_dict[key] = outputs[i].outputs[0].text[1:]
#     else:
#         print("error")

In [23]:
# len(result_dict)

In [24]:
# result_dict

In [25]:
# h011753d9ge86fg41f1g942cg43995abfa69f
# print(english_dict["h011753d9ge86fg41f1g942cg43995abfa69f"])
# print(result_dict["h011753d9ge86fg41f1g942cg43995abfa69f"])
# print(userTranslate_dict_0819["h011753d9ge86fg41f1g942cg43995abfa69f"])

In [26]:
# h2a0fb3f5g008eg41bbg86e1g3319cebc7548
# print(english_dict["h2a0fb3f5g008eg41bbg86e1g3319cebc7548"])
# print(result_dict["h2a0fb3f5g008eg41bbg86e1g3319cebc7548"])
# print(userTranslate_dict_0819["h2a0fb3f5g008eg41bbg86e1g3319cebc7548"])

In [27]:
# h00c363efga48dg4362ga104ge5d6675dc682
# print(english_dict["h00c363efga48dg4362ga104ge5d6675dc682"])
# print(result_dict["h00c363efga48dg4362ga104ge5d6675dc682"])
# print(userTranslate_dict_0819["h00c363efga48dg4362ga104ge5d6675dc682"])

In [28]:
# 'h00c8cc65gc4e9g44f9gbff8gd593fddb2ab9'
# print(english_dict['h00c8cc65gc4e9g44f9gbff8gd593fddb2ab9'])
# print(result_dict['h00c8cc65gc4e9g44f9gbff8gd593fddb2ab9'])
# print(userTranslate_dict_0819['h00c8cc65gc4e9g44f9gbff8gd593fddb2ab9'])

In [29]:
# print(english_dict[test_key[3456]])
# print(result_dict[test_key[3456]])
# print(userTranslate_dict_0819[test_key[3456]])

In [30]:
import pickle

In [31]:
# save data
# with open('../test_data/result_dict.pickle','wb') as fw:
#     pickle.dump(result_dict, fw)

# load data
# with open('../test_data/result_dict.pickle', 'rb') as fr:
#     user_loaded = pickle.load(fr)